# ResNet as a sequence model applied to protein dataset

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Read processed data

In [106]:
dev_df = pd.read_csv('/content/drive/MyDrive/instaDeep/data/dev_filtered.csv')
test_df = pd.read_csv('/content/drive/MyDrive/instaDeep/data/test_filtered.csv')
train_df = pd.read_csv('/content/drive/MyDrive/instaDeep/data/train_filtered.csv')

In [5]:
MAX_SEQ_LEN = 100



---



---



### One-hot Encoding

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

In [8]:
all_seqs = np.concatenate((dev_df['sequence'].values, test_df['sequence'].values, train_df['sequence'].values))

In [9]:
assert len(all_seqs) == len(dev_df) + len(test_df) + len(train_df)

In [10]:
X_all = (''.join(seq) for seq in all_seqs)

In [11]:
X_all = np.array(list(''.join(X_all)), dtype=str)

In [12]:
X_all.shape

(53388595,)

In [13]:
# reshape the sequence data to a shape of N×1 
# to make the OneHotEncoder learn a single encoding 
# that maps each amino-acid symbol to its respective one-hot vector
X_all = X_all.reshape(-1,1)

In [14]:
one_hot_encoder.fit(X_all)

OneHotEncoder(handle_unknown='ignore')

In [17]:
vocabulary_size = len(one_hot_encoder.get_feature_names_out())
vocabulary_size

23

In [18]:
def sequence_to_one_hot_encoding(seq, encoder):
    oht = list(seq)
    oht = np.array(oht).reshape(-1,1)
    oht = encoder.transform(oht).toarray()
    return oht

In [19]:
# idx = np.random.randint(0, 100)
idx = 62
sequence = train_df['sequence'].values[idx]
print(f'Sequence = {sequence}')

matrix = sequence_to_one_hot_encoding(sequence, one_hot_encoder)
print(f'Matrix   = ')
for line in matrix[:5]: # show 5 first lines
    print(line)
print('...')
for line in matrix[-5:]:# and 5 last lines
    print(line)

Sequence = VDIQIFGRTLRINCPDEEKSYLKKASENLEKRLINLKEKSKISNTEQLLFIAALNMCYELDKEKNKFKKYSINIEKCIKLLKNTI
Matrix   = 
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
...
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [20]:
matrix.shape

(85, 23)

In [21]:
len(sequence)

85

In [22]:
def sequence_to_one_hot_encoding(seq, encoder):
    # converting to an array of characters
    seq = np.array(list(seq))

    # generating the one-hot encoding matrix
    oht = encoder.transform(seq.reshape(-1, 1))
    oht = oht.toarray()

    return oht

In [23]:
def one_hot_processing(seq, seq_len):

    # Truncate
    if seq_len > MAX_SEQ_LEN:
        seq = seq[ : MAX_SEQ_LEN]

    # Padding the sequence to the maximum length
    seq = seq.ljust(MAX_SEQ_LEN)

    # One-hot Encoding
    encoded_seq = sequence_to_one_hot_encoding(seq, one_hot_encoder)
    
    # Set padded part to 0s to indicate end of sequence
    if seq_len < MAX_SEQ_LEN:
        encoded_seq[seq_len:] = 0
    
    return encoded_seq

In [24]:
# train_df.iloc[idx][['sequence', 'seq_length']]

test_seq = train_df.iloc[idx]['sequence']
test_seq_len = train_df.iloc[idx]['seq_length']

test_seq_len

85

In [25]:
encoded_test_seq = one_hot_processing(test_seq, test_seq_len)

In [26]:
encoded_test_seq[85]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [27]:
encoded_test_seq[84]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

### Do the encoding 

Apply the one-hot encoding to each sequence in the DataFrame

In [28]:
dev_encoded_seqs = np.array([one_hot_processing(seq, seq_len) 
                            for (seq, seq_len) in zip(dev_df['sequence'], dev_df['seq_length'])])

In [29]:
test_encoded_seqs = np.array([one_hot_processing(seq, seq_len) 
                             for (seq, seq_len) in zip(test_df['sequence'], test_df['seq_length'])])

In [38]:
# train_df = train_df.head(250000)
train_df = train_df.sample(n=250000, random_state=42)

In [39]:
train_encoded_seqs = np.array([one_hot_processing(seq, seq_len) 
                              for (seq, seq_len) in zip(train_df['sequence'], train_df['seq_length'])])

In [40]:
# Save the encoded sequences as a numpy array
np.save('./dev_encoded_seqs.npy', dev_encoded_seqs)
np.save('./test_encoded_seqs.npy', test_encoded_seqs)
np.save('./train_encoded_seqs.npy', train_encoded_seqs)

In [41]:
# Get target labels 
train_labels = train_df["true_label_encoded"].values
test_labels = test_df["true_label_encoded"].values
dev_labels = dev_df["true_label_encoded"].values

### Define the DataLoader

In [32]:
"""
    Define a custom dataset class that will be used to load the data
"""

class ProteinDataset(Dataset):
    def __init__(self, encoded_sequences, labels):
        self.sequences = torch.tensor(encoded_sequences, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.int64)
    
    def __getitem__(self, index):
        # x = self.sequences[index]
        x = self.sequences[index].unsqueeze(0) # add a new dimension for the channel
        y = self.labels[index]
        return x, y

    def __len__(self):
        return len(self.sequences)

In [43]:
train_ds = ProteinDataset(train_encoded_seqs, train_labels)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=128,
    shuffle=True
)

In [44]:
x, y = train_ds[0]

In [45]:
x.shape

torch.Size([1, 100, 23])

In [33]:
dev_ds = ProteinDataset(dev_encoded_seqs, dev_labels)

dev_loader = DataLoader(
    dataset=dev_ds,
    batch_size=256,
    shuffle=False,
)

In [34]:
test_ds = ProteinDataset(test_encoded_seqs, test_labels)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=256,
    shuffle=False,
)

In [21]:
## Test the DataLoader
for batch_ind, (inputs, labels) in enumerate(train_loader):
    break

In [22]:
inputs.shape

torch.Size([32, 1, 100, 23])

## ResNet model as in ProtCNN

In [47]:
import torchvision.models as models

In [48]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [88]:
# Create ResNet18 model with modified layers
model = models.resnet18(pretrained=False)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [89]:
num_classes = 896

In [90]:
len(np.unique(train_labels))

896

In [91]:

# Modify the first convolutional layer to accept input with 100 channels
# model.conv1 = nn.Conv2d(100, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.conv1 = nn.Conv2d(1, 64, kernel_size=8, stride=2, padding=3, bias=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)


# Modify the average pooling layer to output a tensor with shape (1, 1)
model.avgpool = nn.AdaptiveAvgPool2d((1, 1))

# Modify the last fully connected layer of the ResNet18 model 
# by changing the number of output features from the default 1000 to <num_classes>
model.fc = nn.Linear(in_features=512, out_features=num_classes, bias=True)

model.to(device)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [92]:
learning_rate = 0.001

In [93]:
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate,
                             )

In [94]:
learning_rate = optimizer.param_groups[0]['lr']
learning_rate

0.001

In [95]:
# Debugging
# for i, (inputs, labels) in enumerate(tqdm(train_loader)):
#     break

# inputs.shape

# labels.shape

# preds = model(inputs.to(device))

# preds.shape

# preds = torch.argmax(preds.detach(), dim=-1)

# preds.shape

# labels = labels.to(preds.device)

# (preds == labels).sum()

In [96]:
def train(model, dataloader, loss_fn, optimizer):
    """
    Args:
        model  (torch.nn.Module): model.
        dataloader (torch.utils.data.DataLoader): data loader object to use for training.
    Returns:
        loss_total (float): loss value.
        acc_total  (float): accuracy.
    """
    # num_sample: number of samples explored
    num_sample = 0.

    # loss_total, acc_total
    # variables to collect overall loss and accuracy
    loss_total = 0.
    acc_total = 0.
    
    model.train()

    # for inputs, labels in dataloader:
    for i, (inputs, labels) in enumerate(tqdm(dataloader)):
        num_sample += float(inputs.size(0))

        # .zero_grad() to clear the gradients
        optimizer.zero_grad()

        # computing the model predictions
        preds = model(inputs.to(device)) 

        # computing the loss value for the mini-batch
        loss = loss_fn(preds, labels.to(device))

        # computing the gradient w.r.t. to parameters
        loss.backward()

        # update learnable parameters 
        optimizer.step()

        # cummulating loss in loss_total
        loss_total += float(loss.detach().item())

        # cummulating number of correct classifications in acc_total
        acc_total += float(
            (torch.argmax(preds.detach(), dim=-1) == labels.to(preds.device)).sum()
        )
        
        # Display the current progress using tqdm
        if i % 400 == 0:
            train_acc = 100 * acc_total / num_sample
            tqdm.write(f'Batch {i}, Loss: {loss.item():.4f}, Accuracy: {train_acc:.2f}%')

    # dividing by total number of visited samples
    loss_total = loss_total / num_sample
    acc_total = acc_total / num_sample

    return loss_total, acc_total

In [97]:
def evaluate(model, dataloader, loss_fn, optimizer):
    """
    Args:
        model  (torch.nn.Module): model.
        dataloader (torch.utils.data.DataLoader): 
            data loader object to use for training.
    Returns:
        loss_total (float): loss value.
        acc_total  (float): accuracy.
    """
    num_sample = 0.

    loss_total = 0.
    acc_total = 0.

    model.eval()

    for inputs, labels in dataloader:
        num_sample += float(inputs.size(0))

        preds = model(inputs.to(device)).detach()
        loss_total += float(loss_fn(preds, labels.to(device)).detach().item())
        acc_total += float(
            (torch.argmax(preds, dim=-1) == labels.to(preds.device)).sum()
        )

    loss_total = loss_total / num_sample
    acc_total = acc_total / num_sample

    return loss_total, acc_total

In [98]:
max_epochs = 3

for epoch in range(max_epochs):
    train_loss_total, train_acc_total = train(model, train_loader, loss_fn, optimizer)
    val_loss_total, val_acc_total = evaluate(model, dev_loader, loss_fn, optimizer)

    print(f'[EPOCH:{epoch+1:3d}/{max_epochs}]',
        f'train.loss: {train_loss_total:.4f}',
        f'train.acc: {100*train_acc_total:3.2f}%',
        f'val.loss: {val_loss_total:.4f}',
        f'val.acc: {100*val_acc_total:3.2f}%')

  0%|          | 4/1954 [00:00<02:01, 16.01it/s]

Batch 0, Loss: 6.9547, Accuracy: 0.00%


 21%|██        | 403/1954 [00:17<01:07, 23.01it/s]

Batch 400, Loss: 1.2205, Accuracy: 39.68%


 41%|████      | 805/1954 [00:34<00:50, 22.96it/s]

...

Batch 1200, Loss: 0.0871, Accuracy: 97.36%


 82%|████████▏ | 1605/1954 [01:07<00:14, 23.83it/s]

Batch 1600, Loss: 0.1784, Accuracy: 97.24%


100%|██████████| 1954/1954 [01:22<00:00, 23.72it/s]


[EPOCH:  3/3] train.loss: 0.0008 train.acc: 97.19% val.loss: 0.0007 val.acc: 95.63%


In [100]:
test_loss_total, test_acc_total = evaluate(model, test_loader, loss_fn, optimizer)

print(f'[Test set performance]',
        f'test.loss: {test_loss_total:.4f}',
        f'test.acc: {100*test_acc_total:3.2f}%')

[Test set performance] test.loss: 0.0007 test.acc: 95.68%


### Generate predictions

In [114]:
model.eval()

acc_total = 0.

# Keep test predictions
test_preds = np.empty((0,), dtype=np.int64)

for inputs, labels in test_loader:

    preds = model(inputs.to(device)).detach()
    acc_total += float(
        (torch.argmax(preds, dim=-1) == labels.to(preds.device)).sum()
    )

    preds = torch.argmax(preds, dim=-1).cpu().numpy()  # convert to numpy array
    test_preds = np.concatenate([test_preds, preds])


acc_total = acc_total / (len(test_ds))

In [115]:
print(acc_total)

0.956764264432447


In [116]:
len(test_preds)

44639

In [117]:
test_preds[0]

224

In [119]:
test_df['resnet_preds'] = test_preds

In [121]:
test_df.to_csv('/content/drive/MyDrive/instaDeep/data/resnet_test_preds.csv', index=False)